In [2]:
!pip install openai matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [openai]2m7/8 [openai]c]-types]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
import openai
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import time

openai.api_key = ""

data = [
    {"name": "Boho Dress", "desc": "Flowy dress with earthy tones and boho vibes", "colors": ["brown", "tan"], "vibes": ["boho", "relaxed"]},
    {"name": "Urban Jacket", "desc": "Trendy jacket for energetic urban street look", "colors": ["black", "red"], "vibes": ["urban", "energetic"]},
    {"name": "Cozy Sweater", "desc": "Soft warm sweater for cozy winter day", "colors": ["cream", "brown"], "vibes": ["cozy", "warm"]},
    {"name": "Sporty Shoes", "desc": "Comfortable shoes for active sporty lifestyle", "colors": ["white", "blue"], "vibes": ["sporty", "active"]},
    {"name": "Minimal Tote", "desc": "Simple clean tote bag for daily use", "colors": ["white", "black"], "vibes": ["minimal", "modern"]},
    {"name": "Vintage Scarf", "desc": "Retro patterned scarf with classic elegance", "colors": ["burgundy", "gold"], "vibes": ["vintage", "elegant"]},
    {"name": "Athleisure Leggings", "desc": "Stretchy leggings for workout or casual wear", "colors": ["black", "purple"], "vibes": ["athletic", "casual"]},
    {"name": "Chic Blazer", "desc": "Tailored blazer for professional office looks", "colors": ["navy", "beige"], "vibes": ["professional", "chic"]},
    {"name": "Festival Top", "desc": "Colorful crop top with bohemian patterns", "colors": ["orange", "pink"], "vibes": ["festival", "boho"]},
    {"name": "Luxury Watch", "desc": "Elegant watch with minimalist design", "colors": ["gold", "black"], "vibes": ["luxury", "elegant"]}
]